All the files we saved will find
[here](https://drive.google.com/drive/folders/18NjSyrEIbtnT5V56xoC3odkS0TEQuFaw?usp=sharing) .

## Install Neccessary Packages

In [ ]:
!pip install haversine
!pip install nvector

     |████████████████████████████████| 81kB 6.6MB/s 
     |████████████████████████████████| 12.1MB 8.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 225kB 46.6MB/s 
  Created wheel for cartopy: filename=Cartopy-0.19.0.post1-cp37-cp37m-linux_x86_64.whl size=12516162 sha256=8d17f85e9307abd702a002e3ad8e6b9b899d71f4d5b605d35de94261053241d0
  Stored in directory: /root/.cache/pip/wheels/45/7f/3b/37879587817fd1bbbee7b47312e2401b4f542cccf2fbe9b4ee
Successfully built cartopy
  Created wheel for pyshp: filename=pyshp-2.1.3-cp37-none-any.whl size=37325 sha256=9ad6ba2a3a8c669c8c12f5c913708e999d7311d1e86a0385efa284a9a2d9e4ad
  Stored in directory: /root/.cache/pip/wheels/76/2b/d4/53e6b9a0fb0a9f9f29664cf82605af8bc81d5ab44d987896dd
Successfully built pyshp


In [ ]:
import pandas as pd
import numpy as np
from math import radians, degrees, pi, asin, sin, cos, atan2

## Data Loading

In [ ]:
!rm -r /content/sample_data/
!mkdir train valid test

In [ ]:
!curl -o /content/train/SHL-2021-Train.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2021/SHL-2021-Train.zip
!curl -o /content/valid/SHL-2021-Validate_11052021.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2021/SHL-2021-Validate_11052021.zip
!curl -o /content/test/SHL-2021-Test.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2021/SHL-2021-Test.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 83.1M  100 83.1M    0     0  10.7M      0  0:00:07  0:00:07 --:--:-- 11.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.8M  100 10.8M    0     0  8549k      0  0:00:01  0:00:01 --:--:-- 8543k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.6M  100 45.6M    0     0  10.0M      0  0:00:04  0:00:04 --:--:-- 10.0M


In [ ]:
import zipfile
import os
from tqdm import notebook as tqdm
def un_zipFiles(path):
    files=os.listdir(path)
    for file in tqdm.tqdm(files):
        if file.endswith('.zip'):
            filePath=path+'/'+file
            zip_file = zipfile.ZipFile(filePath)
            for names in zip_file.namelist():
                zip_file.extract(names,path)
            zip_file.close()

In [ ]:
un_zipFiles('/content/train')
un_zipFiles('/content/valid')
un_zipFiles('/content/test')
!rm /content/train/SHL-2021-Train.zip /content/valid/SHL-2021-Validate_11052021.zip /content/test/SHL-2021-Test.zip

## Preprocessing

In [ ]:
# Round datetime to nearst integer
def convert_date(time):
  ms=time.strftime('.%f')
  if float(ms)>0.5:
    time=time+pd.Timedelta('1s')
  time1=time.strftime('%Y-%m-%d %H:%M:%S')
  return time1

### Location

In [ ]:
def read_train_data(path):
    location=pd.read_csv(path,delimiter='\t',header=None,names=['location'])
    location=location['location'].str.split(expand=True)
    location.columns=['Timestamp','Ignore2','Ignore3','accuracy(m)','latitude','longitude','altitude']
    location.drop(['Ignore2','Ignore3'],axis=1,inplace=True)
    location['Timestamp']=pd.to_datetime(location.Timestamp, unit='ms')
    location['Timestamp']=pd.to_datetime(location['Timestamp'].apply(lambda x: convert_date(x)))
    location[location.columns[1:]] = location[location.columns[1:]].astype(np.number)
    return location

In [ ]:
# Train
train_location = read_train_data('/content/train/Location.txt')
train_location.head()

,Timestamp,accuracy(m),latitude,longitude,altitude
0,2017-03-25 08:47:38,64.0,50.844494,-0.132922,97.664610
1,2017-03-25 08:47:40,48.0,50.844491,-0.132924,128.267900
2,2017-03-25 08:47:41,48.0,50.844481,-0.132931,135.611425
3,2017-03-25 08:47:42,12.0,50.844469,-0.132930,134.436050
4,2017-03-25 08:47:43,32.0,50.844267,-0.132787,135.993866


In [ ]:
# Valid
val_location = read_train_data('/content/valid/Location.txt')
val_location.head()

,Timestamp,accuracy(m),latitude,longitude,altitude
0,2017-06-14 07:48:24,96.0,50.826394,-0.119515,248.937172
1,2017-06-14 07:48:26,128.0,50.826394,-0.119528,129.362557
2,2017-06-14 07:48:27,96.0,50.826394,-0.119551,130.095681
3,2017-06-14 07:48:28,128.0,50.826396,-0.119589,130.184470
4,2017-06-14 07:48:29,128.0,50.826396,-0.119589,130.184470


In [ ]:
# Test
test_location = read_train_data('/content/test/Location.txt')
test_location.head()

,Timestamp,accuracy(m),latitude,longitude,altitude
0,2049-02-13 13:17:48,350.0,50.866518,-0.087109,205.939654
1,2049-02-13 13:17:49,350.0,50.866518,-0.087109,205.939654
2,2049-02-13 13:17:50,350.0,50.866518,-0.087109,205.939654
3,2049-02-13 13:17:51,350.0,50.866518,-0.087109,205.939654
4,2049-02-13 13:18:39,96.0,50.866653,-0.086119,103.944537


### Cell

In [ ]:
def label(df):
  condition = [df['cellType'] == 'LTE', df['cellType'] == 'GSM', df['cellType'] == 'WCDMA']
  value = [0,1,2]
  df['cellType'] = np.select(condition,value)
  return df

In [ ]:
def read_cell_data(path):
    import gc
    df = pd.read_csv(path,delimiter='\t',header=None,names=['col'])
    df = df['col'].str.split(expand=True, n=14)
    df.drop([1, 2, df.columns[-1]],axis=1,inplace=True)
    temp = df[df.loc[:, 4] == 'GSM'].drop([5, 6, 7, 8, 9, 13], axis=1)
    temp.columns = [col for col in range(len(temp.columns))]
    temp1 = df[df.loc[:, 4] != 'GSM'].drop([5, 6, 7, 8, 9, 10], axis=1)
    temp1.columns = [col for col in range(len(temp1.columns))]
    df = pd.concat([temp, temp1])
    df.columns = ['Timestamp', 'noOfCell', 'cellType', 'asuLevel', 'dBm', 'signalLevel']
    df['Timestamp'] = pd.to_datetime(df.Timestamp, unit='ms')
    df['Timestamp'] = pd.to_datetime(df['Timestamp'].apply(lambda x: convert_date(x)))
    df.drop_duplicates(subset=['Timestamp'], keep='first', inplace=True)
    df[['asuLevel', 'dBm', 'noOfCell', 'signalLevel']] = df[['asuLevel', 'dBm', 'noOfCell', 'signalLevel']].astype(np.float64)
    df = label(df)
    del temp, temp1
    gc.collect()
    return df.sort_index()

In [ ]:
# Train
train_cell = read_cell_data('/content/train/Cells.txt')
train_cell.head()

,Timestamp,noOfCell,cellType,asuLevel,dBm,signalLevel
0,2017-03-25 08:35:23,3.0,0,25.0,-115.0,2.0
2,2017-03-25 08:35:47,13.0,0,20.0,-120.0,1.0
3,2017-03-25 08:35:53,3.0,0,25.0,-115.0,2.0
4,2017-03-25 08:35:59,3.0,0,27.0,-113.0,2.0
5,2017-03-25 08:36:00,3.0,0,27.0,-113.0,2.0


In [ ]:
# Valid
val_cell = read_cell_data('/content/valid/Cells.txt')
val_cell.head()

,Timestamp,noOfCell,cellType,asuLevel,dBm,signalLevel
0,2017-06-14 07:48:16,4.0,0,54.0,-86.0,4.0
2,2017-06-14 07:48:20,1.0,0,54.0,-86.0,4.0
3,2017-06-14 07:48:26,4.0,0,54.0,-86.0,4.0
4,2017-06-14 07:48:29,5.0,0,57.0,-83.0,4.0
6,2017-06-14 07:48:31,5.0,0,57.0,-83.0,4.0


In [ ]:
# Test
test_cell = read_cell_data('/content/test/Cells.txt')
test_cell.head()

,Timestamp,noOfCell,cellType,asuLevel,dBm,signalLevel
0,2049-02-13 13:17:22,1.0,0,72.0,-68.0,4.0
2,2049-02-13 16:17:58,1.0,0,58.0,-82.0,4.0
4,2049-02-15 09:36:14,1.0,0,56.0,-84.0,4.0
6,2049-02-15 09:36:20,1.0,0,58.0,-82.0,4.0
7,2049-02-15 09:36:26,1.0,0,55.0,-85.0,4.0


### GPS

In [ ]:
def read_gps_data(path):
    from tqdm import notebook as tqdm
    with open(path, 'r') as f:
      lines = f.readlines()
      times = []
      no_of_satelites = []
      max_snrs = []
      nearest_satelites = []
      for line in tqdm.tqdm(lines):
         li = line.strip().split()
         max_snr = 0
         nearest_satelite = 0
         for m in range(int(li[-1])):
             snr = float(li[m*4 + 4])
             if snr > max_snr:
                max_snr = snr
                nearest_satelite = int(li[m*4 + 3])
         times.append(li[0])
         no_of_satelites.append(int(li[-1]))
         max_snrs.append(max_snr)
         nearest_satelites.append(nearest_satelite)
    print('------------- Converting to Dataframe -------------')
    gps = pd.DataFrame([times, no_of_satelites, max_snrs, nearest_satelites]).T
    gps.columns = ['Timestamp', 'no_of_satelite', 'max_snr', 'nearest_satelite']
    gps['Timestamp'] = pd.to_datetime(gps.Timestamp, unit='ms')
    gps['Timestamp'] = pd.to_datetime(gps['Timestamp'].apply(lambda x: convert_date(x)))
    gps[['no_of_satelite', 'nearest_satelite', 'max_snr']] = gps[['no_of_satelite', 'nearest_satelite', 'max_snr']].astype(np.float64)
    return gps

In [ ]:
# Train
train_gps = read_gps_data('/content/train/GPS.txt')
train_gps.head()


------------- Converting to Dataframe -------------


,Timestamp,no_of_satelite,max_snr,nearest_satelite
0,2017-03-25 08:35:24,2.0,18.0,7.0
1,2017-03-25 08:35:25,2.0,18.0,7.0
2,2017-03-25 08:35:26,2.0,18.0,7.0
3,2017-03-25 08:35:27,2.0,19.0,9.0
4,2017-03-25 08:35:28,2.0,19.0,9.0


In [ ]:
# Valid
val_gps = read_gps_data('/content/valid/GPS.txt')
val_gps.head()


------------- Converting to Dataframe -------------


,Timestamp,no_of_satelite,max_snr,nearest_satelite
0,2017-06-14 07:48:17,0.0,0.0,0.0
1,2017-06-14 07:48:17,0.0,0.0,0.0
2,2017-06-14 07:48:18,1.0,27.0,168.0
3,2017-06-14 07:48:19,2.0,36.0,17.0
4,2017-06-14 07:48:20,3.0,37.0,17.0


In [ ]:
# Test
test_gps = read_gps_data('/content/test/GPS.txt')
test_gps.head()


------------- Converting to Dataframe -------------


,Timestamp,no_of_satelite,max_snr,nearest_satelite
0,2049-02-13 13:17:23,0.0,0.0,0.0
1,2049-02-13 13:17:24,0.0,0.0,0.0
2,2049-02-13 13:17:25,0.0,0.0,0.0
3,2049-02-13 13:17:26,1.0,22.0,31.0
4,2049-02-13 13:17:27,1.0,22.0,31.0


### WiFi

In [ ]:
def get_first_col(listx):
  return listx[0]

def get_4th(listx):
  return int(listx[3])

def get_rssi_list(l):
  networks = int(l[3])
  rssi_list = list()
  for i in range(networks):
    rssi_list.append(l[i*5+6])
  return rssi_list

#the highest value of rssi(in dB) was selected as best rssi
def get_best_rssi(l):
  networks = int(l[3])
  rssi_list = list()
  best_rssi = 0
  for i in range(networks):
    rssi_list.append(l[i*5+6])
    if i == 0:
      best_rssi = int(l[i*5+6])
    elif best_rssi< int(l[i*5+6]):
      best_rssi = int(l[i*5+6])
  return best_rssi

def level_rssi(l):
  rssi_lessThan50 = 0
  rssi_50_60 = 0
  rssi_60_70 = 0
  rssi_70_80 = 0
  rssi_80_90 = 0
  rssi_zero = 0
  rssi_greaterThan90 = 0

  for items in l:
    if int(items)==0: rssi_zero+=1
    elif int(items)>-50: rssi_lessThan50 += 1
    elif int(items)>-60: rssi_50_60 += 1
    elif int(items)>-70: rssi_60_70 += 1
    elif int(items)>-80: rssi_70_80 += 1
    elif int(items)>-90: rssi_80_90 += 1
    else : rssi_greaterThan90 += 1
  return [rssi_lessThan50, rssi_50_60, rssi_60_70, rssi_70_80, rssi_80_90, rssi_greaterThan90, rssi_zero]

def get_freq(l):
  networks = int(l[3])
  freq_list = list()
  for i in range(networks):
    freq_list.append(round(int(l[i*5+7])/1000))
  return [freq_list.count(2),freq_list.count(5)]


In [ ]:
def read_wifi_data(path):
    import gc
    wifi = pd.read_csv(path, delimiter='\t', header=None, names=['wifi_data'])
    wifi["wifi_data"] = wifi["wifi_data"].str.split(';')
    wifi['Timestamp'] = wifi["wifi_data"].apply(lambda x: get_first_col(x))
    wifi['Timestamp'] = pd.to_datetime(wifi.Timestamp, unit='ms')
    wifi['Timestamp'] = pd.to_datetime(wifi['Timestamp'].apply(lambda x: convert_date(x)))
    wifi['available'] = wifi["wifi_data"].apply(lambda x: get_4th(x))
    wifi["all_rssi"] = wifi["wifi_data"].apply(lambda x: get_rssi_list(x))
    wifi["best_rssi"] = wifi["wifi_data"].apply(lambda x: get_best_rssi(x))
    wifi["rssi_level"] = wifi["all_rssi"].apply(lambda x: level_rssi(x))
    wifi[['excelent_rssi','veryGood_rssi','good_rssi','low_rssi','veryLow_rssi','poor_rssi','No Signal']] = pd.DataFrame(wifi.rssi_level.to_list(), index = wifi.index)
    wifi['frequency'] = wifi["wifi_data"].apply(lambda x: get_freq(x))
    wifi[['2.4GHz','5GHz']] = pd.DataFrame(wifi.frequency.to_list(), index = wifi.index)
    cols = ['Timestamp', 'available', 'best_rssi', 'excelent_rssi', 'veryGood_rssi', 'low_rssi', 'veryLow_rssi', 'poor_rssi', '2.4GHz', '5GHz']
    return wifi[cols]

In [ ]:
# Train
train_wifi = read_wifi_data('/content/train/WiFi.txt')
train_wifi.head()

,Timestamp,available,best_rssi,excelent_rssi,veryGood_rssi,low_rssi,veryLow_rssi,poor_rssi,2.4GHz,5GHz
0,2017-03-25 08:35:23,10,-54,0,3,2,3,1,9,1
1,2017-03-25 08:35:24,10,-54,0,3,2,3,1,9,1
2,2017-03-25 08:35:25,10,-54,0,3,2,3,1,9,1
3,2017-03-25 08:35:26,10,-54,0,3,2,3,1,9,1
4,2017-03-25 08:35:27,10,-54,0,3,2,3,1,9,1


In [ ]:
# Valid
val_wifi = read_wifi_data('/content/valid/WiFi.txt')
val_wifi.head()

,Timestamp,available,best_rssi,excelent_rssi,veryGood_rssi,low_rssi,veryLow_rssi,poor_rssi,2.4GHz,5GHz
0,2017-06-14 07:48:16,11,-36,2,0,2,3,0,11,0
1,2017-06-14 07:48:17,11,-36,2,0,2,3,0,11,0
2,2017-06-14 07:48:18,11,-36,2,0,2,3,0,11,0
3,2017-06-14 07:48:19,11,-36,2,0,2,3,0,11,0
4,2017-06-14 07:48:20,11,-36,2,0,2,3,0,11,0


In [ ]:
# Test
test_wifi = read_wifi_data('/content/test/WiFi.txt')
test_wifi.head()

,Timestamp,available,best_rssi,excelent_rssi,veryGood_rssi,low_rssi,veryLow_rssi,poor_rssi,2.4GHz,5GHz
0,2049-02-13 13:17:22,6,-51,0,1,1,1,0,4,2
1,2049-02-13 13:17:23,6,-51,0,1,1,1,0,4,2
2,2049-02-13 13:17:24,6,-51,0,1,1,1,0,4,2
3,2049-02-13 13:17:25,6,-51,0,1,1,1,0,4,2
4,2049-02-13 13:17:26,6,-51,0,1,1,1,0,4,2


### Labels

In [ ]:
# Train
train_label = pd.read_csv('/content/train/Label.txt',delimiter='\t',header=None,names=['Timestamp','label'])
train_label['Timestamp'] = pd.to_datetime(train_label.Timestamp, unit='ms')
train_label.head()

,Timestamp,label
0,2017-03-25 08:46:23,4
1,2017-03-25 08:46:24,4
2,2017-03-25 08:46:25,4
3,2017-03-25 08:46:26,4
4,2017-03-25 08:46:27,4


In [ ]:
# Valid
val_label = pd.read_csv('/content/valid/Label.txt',delimiter='\t',header=None,names=['Timestamp','label'])
val_label['Timestamp'] = pd.to_datetime(val_label.Timestamp, unit='ms')
val_label.head()

,Timestamp,label
0,2017-06-14 08:04:52,3
1,2017-06-14 08:04:53,3
2,2017-06-14 08:04:54,3
3,2017-06-14 08:04:55,3
4,2017-06-14 08:04:56,3


In [ ]:
# Test
test_label_idx = pd.read_csv('/content/test/Label_idx.txt',delimiter='\n',header=None,names=['Timestamp'])
test_label_idx['Timestamp'] = pd.to_datetime(test_label_idx.Timestamp, unit='ms')
test_label_idx.head()

,Timestamp
0,2049-02-13 13:20:46
1,2049-02-13 13:20:47
2,2049-02-13 13:20:48
3,2049-02-13 13:20:49
4,2049-02-13 13:20:50


### Merge All

In [ ]:
# Train
df = pd.merge(train_label, train_location, on='Timestamp', how='left')
df = pd.merge(df, train_gps, on='Timestamp', how='left')
df = pd.merge(df, train_cell, on='Timestamp', how='left')
df = pd.merge(df, train_wifi, on='Timestamp', how='left')
df.index = np.arange(len(df))
df.drop_duplicates(subset=['Timestamp'], inplace = True)
df.head()

,Timestamp,label,accuracy(m),latitude,longitude,altitude,no_of_satelite,max_snr,nearest_satelite,noOfCell,cellType,asuLevel,dBm,signalLevel,available,best_rssi,excelent_rssi,veryGood_rssi,low_rssi,veryLow_rssi,poor_rssi,2.4GHz,5GHz
0,2017-03-25 08:46:23,4,NaN,NaN,NaN,NaN,4.0,29.0,30.0,NaN,NaN,NaN,NaN,NaN,6.0,-43.0,3.0,0.0,1.0,0.0,1.0,5.0,1.0
1,2017-03-25 08:46:24,4,NaN,NaN,NaN,NaN,4.0,29.0,30.0,NaN,NaN,NaN,NaN,NaN,6.0,-43.0,3.0,0.0,1.0,0.0,1.0,5.0,1.0
2,2017-03-25 08:46:25,4,NaN,NaN,NaN,NaN,4.0,29.0,30.0,NaN,NaN,NaN,NaN,NaN,6.0,-43.0,3.0,0.0,1.0,0.0,1.0,5.0,1.0
3,2017-03-25 08:46:26,4,NaN,NaN,NaN,NaN,4.0,28.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-03-25 08:46:27,4,NaN,NaN,NaN,NaN,4.0,28.0,30.0,NaN,NaN,NaN,NaN,NaN,6.0,-43.0,3.0,0.0,1.0,0.0,1.0,5.0,1.0


In [ ]:
# Valid
df_val = pd.merge(val_label, val_location, on='Timestamp', how='left')
df_val = pd.merge(df_val, val_gps, on='Timestamp', how='left')
df_val = pd.merge(df_val, val_cell, on='Timestamp', how='left')
df_val = pd.merge(df_val, val_wifi, on='Timestamp', how='left')
df_val.index = np.arange(len(df_val))
df_val.drop_duplicates(subset=['Timestamp'], inplace = True)
df_val.head()

,Timestamp,label,accuracy(m),latitude,longitude,altitude,no_of_satelite,max_snr,nearest_satelite,noOfCell,cellType,asuLevel,dBm,signalLevel,available,best_rssi,excelent_rssi,veryGood_rssi,low_rssi,veryLow_rssi,poor_rssi,2.4GHz,5GHz
0,2017-06-14 08:04:52,3,8.0,50.826732,-0.119872,133.989176,16.0,38.0,17.0,NaN,NaN,NaN,NaN,NaN,11.0,-37.0,2.0,0.0,6.0,1.0,0.0,10.0,1.0
1,2017-06-14 08:04:53,3,6.0,50.826738,-0.119856,134.054522,16.0,39.0,17.0,NaN,NaN,NaN,NaN,NaN,11.0,-37.0,2.0,0.0,6.0,1.0,0.0,10.0,1.0
2,2017-06-14 08:04:54,3,6.0,50.826746,-0.119839,134.170604,16.0,39.0,17.0,NaN,NaN,NaN,NaN,NaN,11.0,-37.0,2.0,0.0,6.0,1.0,0.0,10.0,1.0
3,2017-06-14 08:04:55,3,6.0,50.826755,-0.119822,134.297500,16.0,39.0,17.0,NaN,NaN,NaN,NaN,NaN,11.0,-37.0,2.0,0.0,6.0,1.0,0.0,10.0,1.0
4,2017-06-14 08:04:56,3,6.0,50.826761,-0.119806,134.367085,16.0,38.0,17.0,NaN,NaN,NaN,NaN,NaN,11.0,-37.0,2.0,0.0,6.0,1.0,0.0,10.0,1.0


In [ ]:
# Test
df_test = pd.merge(test_label_idx, test_location, on='Timestamp', how='left')
df_test = pd.merge(df_test, test_gps, on='Timestamp', how='left')
df_test = pd.merge(df_test, test_cell, on='Timestamp', how='left')
df_test = pd.merge(df_test, test_wifi, on='Timestamp', how='left')
df_test.index = np.arange(len(df_test))
df_test.drop_duplicates(subset=['Timestamp'], inplace = True)
df_test.head()

,Timestamp,accuracy(m),latitude,longitude,altitude,no_of_satelite,max_snr,nearest_satelite,noOfCell,cellType,asuLevel,dBm,signalLevel,available,best_rssi,excelent_rssi,veryGood_rssi,low_rssi,veryLow_rssi,poor_rssi,2.4GHz,5GHz
0,2049-02-13 13:20:46,NaN,NaN,NaN,NaN,2.0,26.0,164.0,NaN,NaN,NaN,NaN,NaN,6.0,-51.0,0.0,1.0,1.0,1.0,0.0,4.0,2.0
1,2049-02-13 13:20:47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2049-02-13 13:20:48,NaN,NaN,NaN,NaN,2.0,25.0,164.0,NaN,NaN,NaN,NaN,NaN,6.0,-51.0,0.0,1.0,1.0,1.0,0.0,4.0,2.0
4,2049-02-13 13:20:49,NaN,NaN,NaN,NaN,3.0,23.0,164.0,NaN,NaN,NaN,NaN,NaN,6.0,-51.0,0.0,1.0,1.0,1.0,0.0,4.0,2.0
5,2049-02-13 13:20:50,NaN,NaN,NaN,NaN,2.0,22.0,164.0,NaN,NaN,NaN,NaN,NaN,6.0,-51.0,0.0,1.0,1.0,1.0,0.0,4.0,2.0


Save to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df.to_pickle('/content/drive/My Drive/SHL/data/df-v2.pickle')
df_val.to_pickle('/content/drive/My Drive/SHL/data/df_val-v2.pickle')
df_test.to_pickle('/content/drive/My Drive/SHL/data/df_test-v2.pickle')

In [ ]:
df = pd.read_pickle('/content/drive/My Drive/SHL/data/df-v1.pickle')
df_val = pd.read_pickle('/content/drive/My Drive/SHL/data/df_val-v1.pickle')
df_test = pd.read_pickle('/content/drive/My Drive/SHL/data/df_test-v1.pickle')

### Handle NaN

In [ ]:
# Train
# NaN before
df.isna().sum()

Timestamp                0
label                    0
accuracy(m)         319664
latitude            319664
longitude           319664
altitude            319664
no_of_satelite       88988
max_snr              88988
nearest_satelite     88988
noOfCell            113787
cellType            113787
asuLevel            189239
dBm                 189239
signalLevel         189239
available            15884
best_rssi            15884
excelent_rssi        15884
veryGood_rssi        15884
low_rssi             15884
veryLow_rssi         15884
poor_rssi            15884
2.4GHz               15884
5GHz                 15884
dtype: int64

In [ ]:
# Valid
# NaN before
df_val.isna().sum()

Timestamp               0
label                   0
accuracy(m)         59483
latitude            59483
longitude           59483
altitude            59483
no_of_satelite      18606
max_snr             18606
nearest_satelite    18606
noOfCell            45392
cellType            45392
asuLevel            64436
dBm                 64436
signalLevel         64436
available           20587
best_rssi           20587
excelent_rssi       20587
veryGood_rssi       20587
low_rssi            20587
veryLow_rssi        20587
poor_rssi           20587
2.4GHz              20587
5GHz                20587
dtype: int64

In [ ]:
# Test
# NaN before
df_test.isna().sum()

Timestamp                0
accuracy(m)         208108
latitude            208108
longitude           208108
altitude            208108
no_of_satelite       51047
max_snr              51047
nearest_satelite     51047
noOfCell            228467
cellType            228467
asuLevel            262009
dBm                 262009
signalLevel         262009
available            56306
best_rssi            56306
excelent_rssi        56306
veryGood_rssi        56306
low_rssi             56306
veryLow_rssi         56306
poor_rssi            56306
2.4GHz               56306
5GHz                 56306
dtype: int64

In [ ]:
# Train
df.fillna(method='ffill', limit=5, axis=0, inplace=True)
df.fillna(method='bfill', limit=5, axis=0, inplace=True)

# Valid
df_val.fillna(method='ffill', limit=5, axis=0, inplace=True)
df_val.fillna(method='bfill', limit=5, axis=0, inplace=True)

# Test
df_test.fillna(method='ffill', limit=5, axis=0, inplace=True)
df_test.fillna(method='bfill', limit=5, axis=0, inplace=True)

In [ ]:
# Train
# NaN after
df.isna().sum()

Timestamp                0
label                    0
accuracy(m)         301948
latitude            301948
longitude           301948
altitude            301948
no_of_satelite       86467
max_snr              86467
nearest_satelite     86467
noOfCell             98121
cellType             98121
asuLevel            159120
dBm                 159120
signalLevel         159120
available                0
best_rssi                0
excelent_rssi            0
veryGood_rssi            0
low_rssi                 0
veryLow_rssi             0
poor_rssi                0
2.4GHz                   0
5GHz                     0
dtype: int64

In [ ]:
# Valid
# NaN after
df_val.isna().sum()

Timestamp               0
label                   0
accuracy(m)         56969
latitude            56969
longitude           56969
altitude            56969
no_of_satelite      18192
max_snr             18192
nearest_satelite    18192
noOfCell            43325
cellType            43325
asuLevel            60760
dBm                 60760
signalLevel         60760
available           18545
best_rssi           18545
excelent_rssi       18545
veryGood_rssi       18545
low_rssi            18545
veryLow_rssi        18545
poor_rssi           18545
2.4GHz              18545
5GHz                18545
dtype: int64

In [ ]:
# Test
# NaN after
df_test.isna().sum()

Timestamp                0
accuracy(m)         194276
latitude            194276
longitude           194276
altitude            194276
no_of_satelite       49166
max_snr              49166
nearest_satelite     49166
noOfCell            218322
cellType            218322
asuLevel            247454
dBm                 247454
signalLevel         247454
available            46670
best_rssi            46670
excelent_rssi        46670
veryGood_rssi        46670
low_rssi             46670
veryLow_rssi         46670
poor_rssi            46670
2.4GHz               46670
5GHz                 46670
dtype: int64

Don't drop remaining NaN values.

In [ ]:
df.head()

,Timestamp,label,accuracy(m),latitude,longitude,altitude,no_of_satelite,max_snr,nearest_satelite,noOfCell,cellType,asuLevel,dBm,signalLevel,available,best_rssi,excelent_rssi,veryGood_rssi,low_rssi,veryLow_rssi,poor_rssi,2.4GHz,5GHz
0,2017-03-25 08:46:23,4,NaN,NaN,NaN,NaN,4.0,29.0,30.0,NaN,NaN,NaN,NaN,NaN,6.0,-43.0,3.0,0.0,1.0,0.0,1.0,5.0,1.0
1,2017-03-25 08:46:24,4,NaN,NaN,NaN,NaN,4.0,29.0,30.0,NaN,NaN,NaN,NaN,NaN,6.0,-43.0,3.0,0.0,1.0,0.0,1.0,5.0,1.0
2,2017-03-25 08:46:25,4,NaN,NaN,NaN,NaN,4.0,29.0,30.0,NaN,NaN,NaN,NaN,NaN,6.0,-43.0,3.0,0.0,1.0,0.0,1.0,5.0,1.0
3,2017-03-25 08:46:26,4,NaN,NaN,NaN,NaN,4.0,28.0,30.0,NaN,NaN,NaN,NaN,NaN,6.0,-43.0,3.0,0.0,1.0,0.0,1.0,5.0,1.0
4,2017-03-25 08:46:27,4,NaN,NaN,NaN,NaN,4.0,28.0,30.0,NaN,NaN,NaN,NaN,NaN,6.0,-43.0,3.0,0.0,1.0,0.0,1.0,5.0,1.0


## Feature Analysis

### Features based on location

Timewise segmentation for calculating velocity acceleration ...If Time interval between two row greater than 30s/1min/1h  then divide

In [ ]:
def timewise_segmentation(data):
  from tqdm import notebook as tqdm
  dataframe_list=[]
  segmented_dataframe=[]
  for i in tqdm.tqdm(np.arange(len(data)-1)):
    timedelta=pd.to_datetime(data.iloc[i+1]['Timestamp'])-pd.to_datetime(data.iloc[i]['Timestamp'])
    segmented_dataframe.append(data.iloc[i])
    if timedelta< pd.Timedelta('30s'):
      continue
    elif timedelta>= pd.Timedelta('30s'):
      dataframe=pd.DataFrame(segmented_dataframe)
      dataframe_list.append(dataframe)
      segmented_dataframe=[]

  segmented_dataframe.append(data.iloc[-1])
  dataframe=pd.DataFrame(segmented_dataframe)
  dataframe_list.append(dataframe)
    
  return dataframe_list

In [ ]:
train_segmented_list = timewise_segmentation(df)
val_segmented_list = timewise_segmentation(df_val)
test_segmented_list = timewise_segmentation(df_test)

In [ ]:
def convert_to_xy(latitude, longitude):
  import math
  latRad = latitude * (math.pi)/180
  lonRad = longitude * (math.pi)/180
  earthRadius = 6367000
  posX = earthRadius * math.cos(latRad) * math.cos(lonRad)
  posY = earthRadius * math.cos(latRad) * math.sin(lonRad)
  return {'x': posX,'y': posY}
  
def calculate_hsdist_from_pos(loc1,loc2):
  from haversine import Unit
  import haversine as hs
  return hs.haversine(loc1,loc2,unit=Unit.METERS)

def calc_angle(delx,dely,delz):
   import math
   vector_2=[delx,dely,delz]
   if  np.linalg.norm(vector_2)==0:
     return 0
        
   else:
     unit_vector_1 = [0,1,0]
     unit_vector_2 = vector_2 / np.linalg.norm(vector_2)
     dot_product = np.dot(unit_vector_1, unit_vector_2)
     angle = np.arccos(dot_product)*180/np.pi
     
     return angle

In [ ]:
def make_features(list):
    import nvector as nv
    from tqdm import notebook as tqdm
    for data in tqdm.tqdm(list):
        data['distance']=0
        data['velocity']=0
        data['acceleration']=0
        data['vec_x']=0
        data['vec_y']=0
        data['vec_z']=0
        data['vec_azimuth']=0
        data['heading_vector']=0
        data['vel_change_rate']=0
        data.index=np.arange(len(data))

        for i in np.arange(len(data)-1):
            loc1=(float(data.loc[i,'latitude']),float(data.loc[i,'longitude']))
            loc2=(float(data.loc[i+1,'latitude']),float(data.loc[i+1,'longitude']))
            dist=calculate_hsdist_from_pos(loc1,loc2)
            data.loc[i+1,'distance']=dist
            del_t=(pd.to_datetime(data.loc[i+1,'Timestamp'])-pd.to_datetime(data.loc[i,'Timestamp'])).seconds
            v=dist/del_t
            data.loc[i+1,'velocity']=v
            a=(data.loc[i+1,'velocity']-data.loc[i,'velocity'])/del_t
            data.loc[i+1,'acceleration']=a
            wgs84 = nv.FrameE(name='WGS84')
            pointA = wgs84.GeoPoint(latitude=float(data.loc[i,'latitude']), longitude=float(data.loc[i,'longitude']), z=float(data.loc[i,'altitude']), degrees=True)
            pointB = wgs84.GeoPoint(latitude=float(data.loc[i+1,'latitude']), longitude=float(data.loc[i+1,'longitude']), z=float(data.loc[i+1,'altitude']), degrees=True)
            p_AB_N = pointA.delta_to(pointB)
            x, y, z = p_AB_N.pvector.ravel()
            azimuth = p_AB_N.azimuth_deg
            az=float('{0:4.2f}'.format(azimuth))
            data.loc[i+1,'vec_x']=x
            data.loc[i+1,'vec_y']=y
            data.loc[i+1,'vec_z']=z
            data.loc[i+1,'vec_azimuth']=az
            data.loc[i+1,'heading_vector']=calc_angle(x,y,z)
            if data.loc[i,'velocity']!=0:
               data.loc[i+1,'vel_change_rate']=(data.loc[i+1,'velocity']-data.loc[i,'velocity'])/data.loc[i,'velocity']
            elif data.loc[i,'velocity']==0:
               data.loc[i+1,'vel_change_rate']=data.loc[i+1,'velocity']-data.loc[i,'velocity']

In [ ]:
make_features(train_segmented_list)
make_features(val_segmented_list)
make_features(test_segmented_list)

In [ ]:
train=pd.concat([i for i in train_segmented_list], ignore_index = True)
val = pd.concat([i for i in val_segmented_list], ignore_index = True)
test = pd.concat([i for i in test_segmented_list], ignore_index = True)

In [ ]:
train.head()

In [ ]:
val.head()

### Save to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train.to_pickle('/content/drive/MyDrive/SHL/data/train-v2.pickle')
val.to_pickle('/content/drive/MyDrive/SHL/data/val-v2.pickle')
test.to_pickle('/content/drive/MyDrive/SHL/data/test-v2.pickle')

In [ ]:
import pandas as pd
import numpy as np

train = pd.read_pickle('/content/drive/MyDrive/SHL/data/train-v2.pickle')
val = pd.read_pickle('/content/drive/MyDrive/SHL/data/val-v2.pickle')
test = pd.read_pickle('/content/drive/MyDrive/SHL/data/test-v2.pickle')

### Features based on trajectory

In [ ]:
def get_traj(df):
    from tqdm import notebook as tqdm
    ranges = []
    time_of_flights = []
    for row in tqdm.tqdm(range(len(df))):
        range_, time_of_flight = calculateTraf(df.loc[row, 'heading_vector'], df.loc[row, 'velocity'])
        ranges.append(range_)
        time_of_flights.append(time_of_flight)
    traj_df = pd.DataFrame([ranges, time_of_flights]).T
    traj_df.columns = ['range', 'time_of_flight']
    final_df = pd.concat([df, traj_df], axis=1)
    return final_df

In [ ]:
from math import sin,cos,pi

def calculateTraf(angle, velocity ):
	
	#defining gravity
	gravity = float(9.8)
	
	#converting angle to radians
	angle =angle * pi / 180

	#calculating horizontal and vertical components of the velocity
	velocity_h= velocity*cos(angle)
	velocity_v = velocity*sin(angle)
	
	#computing time and distance of flight
	time_of_flight =2 * float(velocity_v) / gravity
	range = float(time_of_flight) * velocity_h
	
	return range, time_of_flight

In [ ]:
train_traj = get_traj(train)
val_traj = get_traj(val)
test_traj = get_traj(test)

### Features based on cell

In [ ]:
def ctype_connection(cell):
    from tqdm import notebook as tqdm
    lte_list_0, gsm_list_0, wcdma_list_0, lte_list_1, gsm_list_1, wcdma_list_1 = [], [], [], [], [], []
    for row in tqdm.tqdm(range(cell.shape[0])):
        lte_0, gsm_0, wcdma_0, lte_1, gsm_1, wcdma_1 = [], [], [], [], [], []
        for col in range(cell.shape[1]-1):
            ctype = cell.loc[row, col]
            if ctype == None:
               continue
            a = cell.loc[row,col+1]
            if (ctype == 'LTE') and (a == '0'):     # if (cell.loc[row,col] == 'LTE/GSM/WCDMA') and (cell.loc[row,col+1] == '0/1') 
               lte_0.append(a)
            elif (ctype == 'GSM') and (a == '0'): 
               gsm_0.append(a)
            elif (ctype == 'WCDMA') and (a == '0'):
               wcdma_0.append(a)
            elif (ctype == 'LTE') and (a == '1'):    
               lte_1.append(a)
            elif (ctype == 'GSM') and (a == '1'): 
               gsm_1.append(a)
            elif (ctype == 'WCDMA') and (a == '1'):
               wcdma_1.append(a)
        lte_list_0.append(len(lte_0))  
        gsm_list_0.append(len(gsm_0))
        wcdma_list_0.append(len(wcdma_0))
        lte_list_1.append(len(lte_1))  
        gsm_list_1.append(len(gsm_1))
        wcdma_list_1.append(len(wcdma_1))
    return [lte_list_0, gsm_list_0, wcdma_list_0, lte_list_1, gsm_list_1, wcdma_list_1]

In [ ]:
def get_connection_features_from_cell(path, df_concat):
    import gc
    df = pd.read_csv(path, delimiter='\t', header=None, names=['col'])
    df = df['col'].str.split(expand=True)
    features = ctype_connection(df)
    cols = ['LTE_no_connection', 'GSM_no_connection', 'WCDMA_no_connection', 'LTE_connection', 'GSM_connection', 'WCDMA_connection']
    cell = pd.DataFrame(features, index=None, columns=None).T
    cell.columns = cols
    cell['Timestamp'] = df.loc[:, 0]
    cell['Timestamp'] = pd.to_datetime(cell.Timestamp, unit='ms')
    cell['Timestamp'] = pd.to_datetime(cell['Timestamp'].apply(lambda x: convert_date(x)))
    cell.fillna(method='ffill', limit=5, axis=0, inplace=True)
    cell.fillna(method='bfill', limit=5, axis=0, inplace=True)
    df_concat = pd.merge(df_concat, cell, on='Timestamp', how='left')
    df_concat.drop_duplicates(subset='Timestamp', inplace=True)
    df_concat.index = np.arange(len(df_concat))
    del cell, df, features
    gc.collect()
    return df_concat

In [ ]:
train_prepared = get_connection_features_from_cell('/content/train/Cells.txt', train_traj)
val_prepared = get_connection_features_from_cell('/content/valid/Cells.txt', val_traj)
test_prepared = get_connection_features_from_cell('/content/test/Cells.txt', test_traj)

### Features based on frequency domain

In [ ]:
def convert_to_frequency_domain(df, cols):
    from scipy import fftpack
    psd_list = []
    for col in cols:
        fft = pd.DataFrame(fftpack.fft(pd.DataFrame(df[col])), columns=['psd_' + col])
        psd_list.append(np.abs(fft) ** 2)
    fft_freq = pd.DataFrame(fftpack.fftfreq(len(psd_list)), columns=['frequency'])
    return pd.concat([df] + [psd for psd in psd_list] + [fft_freq], axis=1)

In [ ]:
train_prepared = convert_to_frequency_domain(train_prepared, train_prepared.columns[2:])
val_prepared = convert_to_frequency_domain(val_prepared, val_prepared.columns[2:])
test_prepared = convert_to_frequency_domain(test_prepared, test_prepared.columns[1:])

In [ ]:
test_prepared.shape, train_prepared.shape, val_prepared.shape

((671172, 76), (980527, 77), (143958, 77))

In [ ]:
test_prepared.head()

### Save to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_prepared.to_pickle('/content/drive/My Drive/SHL/data/train_prepared-v4.pickle')
val_prepared.to_pickle('/content/drive/My Drive/SHL/data/val_prepared-v4.pickle')
test_prepared.to_pickle('/content/drive/My Drive/SHL/data/test_prepared-v4.pickle')

In [ ]:
import pandas as pd
import numpy as np

train_prepared=pd.read_pickle('/content/drive/MyDrive/SHL/data/train_prepared-v2.pickle')
val_prepared=pd.read_pickle('/content/drive/MyDrive/SHL/data/val_prepared-v2.pickle')
test_prepared=pd.read_pickle('/content/drive/MyDrive/SHL/data/test_prepared-v2.pickle')